In [1]:
import networkx as nx
from bs4 import BeautifulSoup
import re
import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt
from tqdm import tqdm_notebook as tqdm
import requests
import os
import numpy as np
from operator import itemgetter
import dill as pickle
import plotly.graph_objects as go 
from collections import Counter
import community
import multiprocessing as mp

In [2]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

In [3]:
dataYears = ["18", "17", "16", "15", "14", "13"]
dataMonths = ["jan", "feb", "mar", "apr", "may", "jun", "jul", "aug", "sep", "oct", "nov", "dec"]
dataMonths2Dig = ["01", "02", "03", "04", "05", "06", "07", "08", "09", "10", "11", "12"]
dataPath = Path("data")
dummyOver400Label = -1
dummyNotGMLabel = -2
ratingsCounterList = []
ratingsCounterListAll = []
gamesPlayedDictList = []
allRatingsPerMonthDict = {}
sampleIDs = {'2020009', '911925', '1503014', '4125029', '608742', '12940690', '13500767', '14105411', '2034387', '24104272'}

monthlyGraphsHashMap = {}
idList = []
nameRatingMap = {}
idStaticDataMap = {}

In [4]:
def monthDescriptiveLabel(cnt):
    yearIndex = cnt // 12
    monthIndex = cnt % 12
    descriptor = dataMonths[monthIndex]+"20"+dataYears[::-1][yearIndex]
    return descriptor

def idxToDate(cnt):
    yearIndex = cnt // 12
    monthIndex = cnt % 12
    descriptor = "20"+dataYears[::-1][yearIndex] + "-" + dataMonths2Dig[monthIndex] + "-01"
    return descriptor
    

#print(monthDescriptiveLabel(33))

In [6]:
def restoreMonthlyGraphHashMap():
    for year in tqdm(dataYears[::-1]):
        for month in tqdm(dataMonths2Dig):
            date = "20" + year + "-" + month + "-01"
            getMonthlyGraph(date)

def restoreIdStaticDataMap():
    with open(os.path.join('MultiGraphsForMonths', 'static_player_data.pickle'), 'rb') as file:
        idStaticDataMap = pickle.load(file)
    
            
restoreMonthlyGraphHashMap()
restoreIdStaticDataMap()

In [ ]:
#from txt to csv, eliminating unnecessary data, after this piece of code is executed, the data for months is situated in aptly named folders dec13, jan14 etc.

def dataPreprocessing():
    dataMonthsRev = list(reversed(dataMonths)) # reversing the list because pre august 2016 data had a different organisation within the file
    changeInDataset = True
    ratingLimit = 2000
    ratingsCounterList.clear()
    ratingsCounterListAll.clear()
    
    for year in dataYears[::-1]:
        for month in dataMonths:
            
            ratingsCounter = Counter()
            ratingsCounterAll = Counter()
            
            folderName = month + year
            if folderName == "sep16":
                changeInDataset = False

            fileName = "standard_" + folderName + "frl.txt"
            with open(dataPath/folderName/fileName, 'r') as myfile:
                with open(dataPath/folderName/(folderName + ".csv"), 'w') as writeFile:
                    people = []
                    lines = myfile.readlines()
                    for line in tqdm(lines):
                        id = line[:15].strip()
                        
                        if id == 'ID Number': # eliminate file headers
                            continue
                        
                        name = line[15:76].strip()
                        name = name.replace(',', ';', 10) # replace all commas in name with semicolons, because comma is a delimiter in csv file
                        
                        nat = line[76:80].strip()
                        gender = line[80]
                        
                        if changeInDataset:
                            rating_str = line[109:115].strip()
                        else:
                            rating_str = line[113:118].strip()
                        
                        if len(rating_str) > 0:
                            rating = int(rating_str)
                            
                        if changeInDataset:
                            games_played = line[115:118].strip()
                        else:
                            games_played = line[119:122].strip()
            
#                         if len(games_played_str) > 0:
#                             games_played = int(games_played_str)
                        
                        if changeInDataset:
                            birth = line[122:126].strip()
                        else:
                            birth = line[126:130].strip()
                            
#                         if len(birth_str) > 0:
#                             birth = int(birth_str)
                        
                        if changeInDataset:
                            activity = line[128:].strip()
                        else:
                            activity = line[132:].strip()

                        if 'i' not in activity and len(rating_str) > 0 and rating > ratingLimit: # appending every entry to a list for sorting later
                            people.append((id, name, nat, rating_str, gender, games_played, birth, activity))
                            if rating > 2400 and id not in idList:
                                idList.append(id)
                                idStaticDataMap[id] = { 'name': name, 'nat': nat, 'gender': gender, 'birth': birth, 'activity': activity }
                                allRatingsPerMonthDict[id] = [None] * 72                    
                                
                            if id in idList:
                                nameRatingMap[(name, rating_str)] = id
                            if rating > 2400:
                                ratingsCounter[rating] += 1
                                monthIndex = dataMonths.index(month) + 12 * dataYears.index(year)
                                allRatingsPerMonthDict[id][monthIndex] = rating
                            ratingsCounterAll[rating] += 1

                    
                    people.sort(key=lambda tup: int(tup[0])) # sort all people by their FIDE identification 
                    for entry in people:
                        writeFile.write(entry[0] + "," + entry[1] + "," + entry[2] + "," + entry[3] + "," + entry[4] + "," + entry[5] + "," + entry[6] + "," + entry[7] + '\n')

            ratingsCounterList.append(ratingsCounter)
            ratingsCounterListAll.append(ratingsCounterAll)
    with open(os.path.join('MultiGraphsForMonths', 'static_player_data.pickle'), 'wb') as file:
        pickle.dump(idStaticDataMap, file)
#ratings in the constructed txt files are for the end of the month and should be used for building the next month's graph
dataPreprocessing()

In [ ]:
# plotting graphs like a madman

def plotRatingDistributions(all = False):
    fig = go.Figure()
    i = 0
    ratingsToIterateOver = []
    if (all):
        ratingsToIterateOver = ratingsCounterListAll
    else:
        ratingsToIterateOver = ratingsCounterList
        
    for rc in ratingsToIterateOver:
        rc = sorted(rc.items(), key=itemgetter(0))
        fig.add_trace(go.Scattergl(x = [xc for xc, _ in rc], y = [yc for _, yc in rc] ,
                    mode='lines+markers',
                    name=monthDescriptiveLabel(i)))
        i = i + 1
    fig.show() 

In [ ]:
plotRatingDistributions(True)

In [ ]:
plotRatingDistributions(False)

In [ ]:
def plotRatingsForSamplePlayers(sampleIDs):
    fig = go.Figure()
    for playerID in sampleIDs:
        ratingsArray = allRatingsPerMonthDict[playerID]
        fig.add_trace(go.Scattergl(x = [monthDescriptiveLabel(i) for i in range(72)], y = list(ratingsArray) ,
                    mode='lines+markers',
                    name = playerID,
                    connectgaps=False))
    fig.show()

plotRatingsForSamplePlayers(sampleIDs)

In [ ]:
def plotMatchesNumberByMonth():
    fig = go.Figure()
    y_list = []
    for G in monthlyGraphsHashMap.values():
        y_list.append(len(G.edges()))
    fig.add_trace(go.Scattergl(x = [monthDescriptiveLabel(i) for i in range(72)], y = y_list ,
                    mode='lines+markers',
                    connectgaps=False))
    fig.show()
    
plotMatchesNumberByMonth()
        

In [ ]:
from collections import defaultdict

def plotMatchesNumberForSamplePlayers(sampleIDs):
    fig = go.Figure()
    
    matchesDict = defaultdict(list)
    #for pid in sampleIDs:
    #    matchesDict[pid] = []
    
    for year in dataYears[::-1]:
        for month in dataMonths2Dig:
            date = "20" + year + "-" + month + "-01"
            G = getMonthlyGraph(date)
            for playerID in sampleIDs:
                matchesNum = len(G.edges(playerID))
                matchesDict[playerID].append(matchesNum)
            
        
    for playerID in sampleIDs:
        fig.add_trace(go.Scattergl(x = [monthDescriptiveLabel(i) for i in range(72)], y = matchesDict[playerID] ,
                    mode='lines+markers',
                    name = idStaticDataMap[playerID]['name'],
                    connectgaps=False))
    fig.show()
    
plotMatchesNumberForSamplePlayers(sampleIDs)

In [ ]:
def buildNonIsolateNetwork(G):
    GNonZeroDeg = nx.Graph() #graph containing only the non-zero degree subset of vertices of G and all the edges between them
    isolateList = nx.isolates(G)
    #print("Broj izolovanih: ", len(list(isolateList)))
    #print(*list(isolateList), sep='\n')
    GNonZeroDeg = G.copy()
    GNonZeroDeg.remove_nodes_from(isolateList)
    return GNonZeroDeg

''' 
    N: number of nodes, 
    M: number of links, 
    ⟨k⟩: mean degree, 
    ⟨k2⟩/⟨k⟩: heterogeneity parameter, 
    C: clustering coefficient (Newman) and 
    C WS : clustering coefficient (Watts-Strogatz), 
    r: assortativity parameter, 
    Q: modularity, 
    μ: mixing parameter and 
    N C : number of communities. 
    The values of Q, μ and N C were obtained by employing the Louvain algorithm. 
    Values between parentheses correspond to randomized networks using the double-edge swapping method. 
'''
output = mp.Queue()
def calculateMetricsForMonth(year, month, output):
    date = "20" + year + "-" + month + "-01"
    G = getMonthlyGraph(date)
    N = len(G.nodes)
    M = len(G.edges())
    k = 2 * M / N
    if M==0:
        print("No matches for "+date)
        return None
    heterogenity_parameter = sum(map(lambda y: y[1] * y[1], G.degree(G.nodes))) / (2 * M)
    G2 = multiGraphToGraph(G)
    C = np.mean(list(nx.clustering(G2).values()))
    r = nx.degree_assortativity_coefficient(G)
    #             r['pearson'] = nx.degree_pearson_correlation_coefficient(G)
    Q = nx.algorithms.community.greedy_modularity_communities(G)
    Q1 = community.modularity(community.best_partition(G2), G2)
    #             G3 = buildNonIsolateNetwork(G2)
    #             Q_non_isolate = community.modularity(community.best_partition(G3), G3)
    
    output.put((year, month, N, M, k, heterogenity_parameter, C, r, Q, Q1))


def calculateMetrics():
    for year in dataYears[::-1]:
        for month in dataMonths2Dig[:1]:
            calculateMetricsForMonth(year, month)


def calculateMetricsParallel():
#     process_pool = mp.Pool(processes=8)
    processes = []
    
    for year in dataYears[::-1]:
        for month in dataMonths2Dig:
            processes.append(mp.Process(target=calculateMetricsForMonth, args=(year, month, output)))
#     result_list = process_pool.map(calculateMetricsForMonth,
#                            ((year, month) for year in dataYears[::-1] for month in dataMonths2Dig))

#     for res in result_list:
#         print(res[7])
        
#     process_pool.close()
#     process_pool.join()

    for p in processes:
        p.start()

    # Exit the completed processes
    for p in processes:
        p.join()

    # Get process results from the output queue
    results = [output.get() for p in processes]
    print(results)
            
# calculateMetricsParallel()
# calculateMetricsForMonth("15", "01", output)
print(output.get())

In [9]:
def multiGraphToGraph(G):
    G2 = nx.Graph()
    G2.add_nodes_from(list(G))
    
    for u,v,data in G.edges(data=True):
        if G2.has_edge(u,v):
            G2[u][v]['weight'] += 1
        else:
            G2.add_edge(u, v, weight=1)
            
    # nabudziti da ima sve cvorove; furthermore proveriti zasto ne radi
    
    return G2

def weightedEdgeCompose(A, B):
    incrementalAggregate = nx.compose_all([A,B])
    for edge in A.edges():
        src = edge[0]
        dst = edge[1]
        target = list((u,v) for u,v,d in B.edges(data = True) if u == src and v == dst)
        if len(target) > 0:
            incrementalAggregate[src][dst]['weight']+=A[src][dst]['weight']    
    return incrementalAggregate

def composeLoop(graphs):
    A = graphs[0]
    for i in range(1,len(graphs)):
        B = graphs[i]
        A = weightedEdgeCompose(A, B)
    return A

def makeAggregation(idxList):
    return composeLoop([multiGraphToGraph(getMonthlyGraph(idxToDate(idx))) for idx in tqdm(idxList)])

def plotIncrementalAggregation(idxList):
    incrementalList = []
    Q = []
    fig = go.Figure()
    
    for idx in tqdm(idxList):
        incrementalList.append(idx)
        
        a = makeAggregation(incrementalList)
        nx.write_gml(a, "temp-agregate-graph" + str(idx) + ".gml")
        Q.append(community.modularity(community.best_partition(a), a))
    fig.add_trace(go.Scattergl(x = [monthDescriptiveLabel(i) for i in idxList], y = Q,
                mode='lines+markers',
#                 name = idStaticDataMap[playerID]['name'],
                connectgaps=False))
    
    fig.show()
    
# plotIncrementalAggregation(range(12))

In [ ]:
rjecnikrjecnika = {}
for fideID in idList:
    rjecnikrjecnika[fideID] = {}
#fetching matches of a particular player commited for the specified month (many of these mathes were, possibly, played several months before, howbeit, were only commited now)

#id = FIDE id, date = string in yyyy-mm-01 format, returns adjacency list in form ("surname, name", "FIDE rating")
def fetchMatches(id, date):
    targetLink = "https://ratings.fide.com/individual_calculations.phtml?idnumber="+str(id)+"&rating_period="+date
    #page = requests.get("https://ratings.fide.com/individual_calculations.phtml?idnumber=2020009&rating_period=2015-11-01")
    page = requests.get(targetLink)
    soup = BeautifulSoup(page.content, 'html.parser')
    adjacencyDict = {}

    #fetch tournaments table "by rope and stick"
    tournament_table_tbody = soup.find('tr',  {"bgcolor": "#CC9966"})
    if tournament_table_tbody is None:
        return None
    tournament_table_tbody = tournament_table_tbody.parent
    
    rows = tournament_table_tbody.findAll("tr")
    tournament_rating = 0
    tournament_date = ''
    for tr in rows:
        if tr.has_attr('bgcolor'):
            if tr['bgcolor']=="#CC9966":               
                #for each tournament get month when it started
                tournament_date = tr.findAll('td')[3].contents[0].strip()
                tournament_date = tournament_date[:-2]+"01"
            if tr['bgcolor']=="#e6e6e6":
                tournament_rating = tr.findAll("td")[1].contents[0].strip()
                
        
        tdList = tr.findAll("td", {"class": "list4"})
        for td in tdList:
            if td.img:
                img_src = td.img.get('src')
                #print(img_src)
                if img_src == "/imga/clr_bl.gif":
                    color = "b"
                else: 
                    color = "w"
                td.img.decompose()

                cnt = 1
                for child in td.children:
                #print("Child ", cnt, ": ", child)
                    if cnt == 1:
                        name = child.strip()
                    if cnt == 4:
                        if child.font: #if * exists in the rating
                            rescaling = 1
                        else:
                            rescaling = 0
                        rating = child.contents[0].strip()
                    if cnt == 6:
                        result = child.contents[0].strip()
                    if cnt == 8:
                        transfer = child.contents[0].strip()
                    if cnt == 11:
                        if tournament_date in adjacencyDict.keys():
                            adjacencyDict[tournament_date].append((name, rating, tournament_rating, rescaling, color, result, transfer))
                        else:
                            adjacencyDict[tournament_date] = [(name, rating, tournament_rating, rescaling, color, result, transfer)]
                        
                        if rjecnikrjecnika[id].get(tournament_date) is None:
                            rjecnikrjecnika[id][tournament_date] = [(name, rating, tournament_rating, rescaling, color, result, transfer)]
                        else: 
                            rjecnikrjecnika[id][tournament_date].append((name, rating, tournament_rating, rescaling, color, result, transfer))
                    cnt = cnt + 1
    
    return adjacencyDict
    
# print(fetchMatches("2020009", "2015-11-01"))
# print(fetchMatches("1401815", "2018-01-01"))


In [5]:
def getMonthlyGraph(date):
    if date in monthlyGraphsHashMap.keys():
        return monthlyGraphsHashMap[date]
    elif os.path.exists(os.path.join("MultiGraphsForMonths", date + "-graph.gml")):
        G = nx.read_gml(os.path.join("MultiGraphsForMonths", date + "-graph.gml"))
        monthlyGraphsHashMap[date] = G
        return G
    else:
        G = nx.MultiGraph()
        for fideID in idList:
            G.add_node(str(fideID))
            
#         G.add_node(str(dummyOver400Label))
#         G.add_node(str(dummyNotGMLabel))

        monthlyGraphsHashMap[date] = G
        return G
    
def saveMonthlyGraphs():
    for k, v in monthlyGraphsHashMap.items():
        nx.write_gml(v, os.path.join("MultiGraphsForMonths", k + "-graph.gml"))


def existsOppositeEdge(G, fideID, opponentID, eloGain, result, color):
    edges = G.get_edge_data(opponentID, fideID)
#     print(edges)
    if edges is not None:
        for _, attr in edges.items():
    #         print('prvi', attr['edgeSrc'] == opponentID)
    #         print('drugi', float(attr['ratingGainWhite']) == - float(eloGain))
    #         print('treci', float(attr['resultWhite']) == 1.0 - float(result))
    #         print('cetvrti', ((attr['whiteLabel'] == str(opponentID) and color == 'b') or (attr['whiteLabel'] == str(fideID) and color == 'w')))
            if attr['edgeSrc'] == opponentID and float(attr['ratingGainWhite']) == - float(eloGain) and float(attr['resultWhite']) == 1.0 - float(result) and \
            ((attr['whiteLabel'] == str(opponentID) and color == 'b') or (attr['whiteLabel'] == str(fideID) and color == 'w')):
                return True
        
    return False  
    
    
def buildMonthlyGraph(date):
    for fideID in tqdm(idList):
        gamesDict = fetchMatches(fideID, date)
    
        if gamesDict is not None:
            for month in gamesDict:
                G = getMonthlyGraph(month)
                for game in gamesDict[month]:
                    opponentName = game[0]
                    opponentRating = game[1]
                    opponentID = nameRatingMap.get((opponentName.replace(',', ';', 10), opponentRating))
                    rating = game[2]
                    diffOver400 = game[3]
                    color = game[4]
                    result = game[5]
                    eloGain = game[6]

                    if opponentID is None and diffOver400 == 1: # we take into consideration where 
                        continue
#                         opponentID = dummyOver400Label
                        
                    if opponentID is None and diffOver400 == 0: # edges where one player has the rating 2400+ and other is below 2400, we do not take into consideration
                        continue

                    if color == 'w':
                        if not existsOppositeEdge(G, str(fideID), str(opponentID), eloGain, result, color):
                            G.add_edge(str(fideID), str(opponentID), ratingGainWhite = eloGain, whiteLabel = str(fideID), blackLabel = str(opponentID), resultWhite = result, edgeSrc = str(fideID))
                    else:
                        if not existsOppositeEdge(G, str(fideID), str(opponentID), str(-float(eloGain)), str(1.0 - float(result)), color):
                            G.add_edge(str(fideID), str(opponentID), ratingGainWhite = str(-float(eloGain)), whiteLabel = str(opponentID), blackLabel = str(fideID), resultWhite = str(1.0 - float(result)), edgeSrc = str(fideID))
                
def buildMonthlyGraphOffline(date):
    G = getMonthlyGraph(date)
    for fideID in tqdm(idList):
        gamesDict = rjecnikrjecnika.get(fideID).get(date)
        
        if gamesDict is not None:
            for game in gamesDict:
                opponentName = game[0]
                opponentRating = game[1]
                opponentID = nameRatingMap.get((opponentName.replace(',', ';', 10), opponentRating))
                rating = game[2]
                diffOver400 = game[3]
                color = game[4]
                result = game[5]
                eloGain = game[6]

                if opponentID is None and diffOver400 == 1: # we take into consideration where 
                    continue
    #                         opponentID = dummyOver400Label

                if opponentID is None and diffOver400 == 0: # edges where one player has the rating 2400+ and other is below 2400, we do not take into consideration
                    continue

                if color == 'w':
                    if not existsOppositeEdge(G, str(fideID), str(opponentID), eloGain, result, color):
                        G.add_edge(str(fideID), str(opponentID), ratingGainWhite = eloGain, whiteLabel = str(fideID), blackLabel = str(opponentID), resultWhite = result, edgeSrc = str(fideID))
                else:
                    if not existsOppositeEdge(G, str(fideID), str(opponentID), str(-float(eloGain)), str(1.0 - float(result)), color):
                        G.add_edge(str(fideID), str(opponentID), ratingGainWhite = str(-float(eloGain)), whiteLabel = str(opponentID), blackLabel = str(fideID), resultWhite = str(1.0 - float(result)), edgeSrc = str(fideID))

def buildAllMonthlyGraphs():
    folderName = "MultiGraphsForMonths"
    if not os.path.exists(folderName):
        os.makedirs(folderName)
    for year in dataYears[::-1]:
        for month in dataMonths2Dig:
            buildMonthlyGraph("20" + year + '-' + month + '-01')
            saveMonthlyGraphs()
            with open(os.path.join('MultiGraphsForMonths', 'crawler.pickle'), 'wb') as file:
                pickle.dump(rjecnikrjecnika, file)
            print('Done with', year, month)
            
def buildAllMonthlyGraphsOffline():
    folderName = "MultiGraphsForMonths"
    if not os.path.exists(folderName):
        os.makedirs(folderName)
        
    with open(os.path.join('MultiGraphsForMonths', 'crawler.pickle'), 'rb') as file:
        global rjecnikrjecnika
        rjecnikrjecnika = pickle.load(file)
            
    for year in dataYears[::-1]:
        for month in dataMonths2Dig:
            buildMonthlyGraphOffline("20" + year + '-' + month + '-01')
#             saveMonthlyGraphs()
            print('Done with', year, month)
    saveMonthlyGraphs()

In [ ]:
buildAllMonthlyGraphsOffline()

In [ ]:
def buildAllMonthlyGraphsTest(yearsToDo):
    for year in yearsToDo:
        for month in dataMonths2Dig:
            buildMonthlyGraph("20" + year + '-' + month + '-01')
            
            folderName = "MultiGraphsForMonths"
            if not os.path.exists(folderName):
                os.makedirs(folderName)
            saveMonthlyGraphs()
            with open(os.path.join('MultiGraphsForMonths', 'crawler.pickle'), 'wb') as file:
                pickle.dump(rjecnikrjecnika, file)
            print('Done with', year, month)
            
#buildAllMonthlyGraphsTest(["13", "14", "15"])
#buildAllMonthlyGraphsTest(["16", "17", "18"])

In [ ]:
#with open(os.path.join('MultiGraphsForMonths', 'crawler.pickle'), 'rb') as file:
#    rjecnikrjecnika = pickle.load(file)
#buildAllMonthlyGraphsTest(["16", "17", "18"])